In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [2]:
df= pd.read_csv('/content/Netflix Userbase.csv')
df.head()

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
0,1,Basic,10,15/01/22,10/6/2023,United States,28,Male,Smartphone,1 Month
1,2,Premium,15,5/9/2021,22/06/23,Canada,35,Female,Tablet,1 Month
2,3,Standard,12,28/02/23,27/06/23,United Kingdom,42,Male,Smart TV,1 Month
3,4,Standard,12,10/7/2022,26/06/23,Australia,51,Female,Laptop,1 Month
4,5,Basic,10,1/5/2023,28/06/23,Germany,33,Male,Smartphone,1 Month


In [3]:
# Create a DataFrame
df = pd.DataFrame(df)

# Function to clean and standardize dates
def clean_date(date_str):
    if '/' in date_str or '-' in date_str:
        for sep in ['/', '-']:
            if sep in date_str:
                parts = date_str.split(sep)
                if len(parts[-1]) == 2:  # Expand 2-digit years
                    year = int(parts[-1])
                    year += 2000 if year < 50 else 1900
                    parts[-1] = str(year)
                return sep.join(parts).replace('-', '/')
    return date_str.replace('-', '/')

# Apply cleaning and convert to DD/MM/YY format
df["Join Date"] = pd.to_datetime(df["Join Date"].apply(clean_date), format='%d/%m/%Y').dt.strftime('%d/%m/%y')
df["Last Payment Date"] = pd.to_datetime(df["Last Payment Date"].apply(clean_date), format='%d/%m/%Y').dt.strftime('%d/%m/%y')


In [4]:
df.drop('Plan Duration', axis =1)

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device
0,1,Basic,10,15/01/22,10/06/23,United States,28,Male,Smartphone
1,2,Premium,15,05/09/21,22/06/23,Canada,35,Female,Tablet
2,3,Standard,12,28/02/23,27/06/23,United Kingdom,42,Male,Smart TV
3,4,Standard,12,10/07/22,26/06/23,Australia,51,Female,Laptop
4,5,Basic,10,01/05/23,28/06/23,Germany,33,Male,Smartphone
...,...,...,...,...,...,...,...,...,...
2495,2496,Premium,14,25/07/22,12/07/23,Spain,28,Female,Smart TV
2496,2497,Basic,15,04/08/22,14/07/23,Spain,33,Female,Smart TV
2497,2498,Standard,12,09/08/22,15/07/23,United States,38,Male,Laptop
2498,2499,Standard,13,12/08/22,12/07/23,Canada,48,Female,Tablet


In [ ]:
#1 What is the breakdown of users by subscription type?

frequency = df['Subscription Type'].value_counts()
percentage = df['Subscription Type'].value_counts()*100/len(df)
result = pd.concat([frequency, percentage], axis=1)
print("Frequency and Percentage Distribution of Subscription Type:")
result.columns = ['Frequency', 'Percentage']
result

Frequency and Percentage Distribution of Subscription Type:


,Frequency,Percentage
Subscription Type,,
Basic,999,39.96
Standard,768,30.72
Premium,733,29.32


In [ ]:
#2 How does Age correlate with Monthly Revenue?

df['Monthly Revenue'].corr(df['Age'])

-0.021143264071447447

In [ ]:
#3 What is the distribution of users across different countries?

Distribution = df['Country'].value_counts()
print(Distribution)

Country
United States     451
Spain             451
Canada            317
United Kingdom    183
Australia         183
Germany           183
France            183
Brazil            183
Mexico            183
Italy             183
Name: count, dtype: int64


In [32]:
#4 Gender Distribution

Gender = (df['Gender'].value_counts())
dist = (Gender / len(df)) * 100
print(dist)

Gender
Female    50.28
Male      49.72
Name: count, dtype: float64


In [ ]:
#5 What is the gender distribution across subscription type?

Gender_by_Sub = df.groupby('Subscription Type')['Gender'].value_counts().unstack(fill_value=0)
print(Gender_by_Sub)

Gender             Female  Male
Subscription Type              
Basic                 512   487
Premium               364   369
Standard              381   387


In [ ]:
#6 How does monthly revenue vary by subscription type?

total_revenue = df['Monthly Revenue'].sum()
revenue_by_type = df.groupby('Subscription Type')['Monthly Revenue'].sum()
percentage_by_type = (revenue_by_type / total_revenue) * 100

result = pd.concat([revenue_by_type, percentage_by_type], axis=1)
result.columns = ['Total Revenue', 'Percentage']

result

,Total Revenue,Percentage
Subscription Type,,
Basic,12469,39.874005
Premium,9229,29.512967
Standard,9573,30.613028


In [38]:
#7 Which country contributes the highest revenue

Top = df.groupby('Country')['Monthly Revenue'].sum().idxmax()
print(f"The country with the highest revenue is {Top} with the sum of 5664.")

The country with the highest revenue is United States with the sum of 5664.


In [20]:
#8 Which subscription type is the least popular

Sub = df['Subscription Type'].value_counts()
least_popular_sub = Sub.idxmin()
print(f"The least popular subscription is {least_popular_sub}")

The least popular subscription is Premium


In [ ]:
#9 What is the percentage of users using each device

Device_counts = df['Device'].value_counts()
Device_percentages = (Device_counts / len(df)) * 100
print(Device_percentages)

Device
Laptop        25.44
Tablet        25.32
Smartphone    24.84
Smart TV      24.40
Name: count, dtype: float64


In [ ]:
#10 Which Gender generates the highest revenue?

Gender = df.groupby('Gender')['Monthly Revenue'].sum()
percentage_by_gender = (Gender / Gender.sum()) * 100
highest_revenue_gender = Gender.idxmax()
print(f"The {highest_revenue_gender} has the highest revenue with a percentage of {percentage_by_gender[highest_revenue_gender]:.2f}%")

The Female has the highest revenue with a percentage of 50.32%


In [ ]:
#11 What is the average monthly revenue per user?

User = df['Monthly Revenue'].mean()
print(f"The average monthly revenue per user is {User}")

The average monthly revenue per user is 12.5084


In [ ]:
#12 What is the average time between the join date and the last payment date?

df['Join Date'] = pd.to_datetime(df['Join Date'], format='%d/%m/%y', errors='coerce')
df['Last Payment Date'] = pd.to_datetime(df['Last Payment Date'], format='%d/%m/%y', errors='coerce')
df['Time Gap'] = (df['Last Payment Date'] - df['Join Date']).dt.days/30
average_time_gap = df['Time Gap'].mean()
print(f"The average time between join date and last payment date is: {average_time_gap} months")

The average time between join date and last payment date is: 10.287306666666666 months


In [ ]:
#13 What device(s) are most commonly used by users?

Device = df['Device'].value_counts().idxmax()
print(f"The most commonly used device is {Device}.")

The most commonly used device is Laptop.


In [34]:
#14 Is there a relationship between device usage and subscription type?

Relationship = pd.crosstab(df['Device'], df['Subscription Type'])
chi2, p, _, _ = chi2_contingency(Relationship)
print(Relationship)
print(f"\nChi-square statistic: {chi2}")
print(f"P-value: {p}")

if p < 0.05:
    print("Conclusion: There is a significant association between device and subscription type.")
else:
    print("Conclusion: There is no significant association between device and subscription type.")

Subscription Type  Basic  Premium  Standard
Device                                     
Laptop               259      192       185
Smart TV             238      188       184
Smartphone           251      171       199
Tablet               251      182       200

Chi-square statistic: 2.777177373594449
P-value: 0.836248659902357
Conclusion: There is no significant association between device and subscription type.


In [ ]:
#15 Is there a relationship between gender and subscription type?

Relationship = pd.crosstab(df['Gender'], df['Subscription Type'])
chi2, p, _, _ = chi2_contingency(Relationship)
print(Relationship)
print(f"\nChi-square statistic: {chi2}")
print(f"P-value: {p}")

if p < 0.05:
    print("Conclusion: There is a significant association between gender and subscription type.")
else:
    print("Conclusion: There is no significant association between gender and subscription type.")

Subscription Type  Basic  Premium  Standard
Gender                                     
Female               512      364       381
Male                 487      369       387

Chi-square statistic: 0.6282267388216123
P-value: 0.7304362143419852
Conclusion: There is no significant association between device and subscription type.


In [ ]:
#16 What age group contribute the most revenue?

age_revenue = df.groupby('Age')['Monthly Revenue'].sum()
max_revenue_age = age_revenue.idxmax()
max_revenue_sum = age_revenue.max()

print(f"The age with the highest Monthly Revenue is {max_revenue_age} with the sum of {max_revenue_sum}.")

The age with the highest Monthly Revenue is 30 with the sum of 1475.


In [6]:
#17 What device is associated with premium subscription

Premium= df[df['Subscription Type'] == 'Premium']

most_common_device = Premium['Device'].value_counts().idxmax()

print(f"The device type most commonly associated with Premium subscription is: {most_common_device}")

The device type most commonly associated with Premium subscription is: Laptop


In [ ]:
#18 How has the user base grown over time?

df['Join Date'] = pd.to_datetime(df['Join Date'])
df['Year'] = df['Join Date'].dt.year
user_growth_by_year = df.groupby('Year')['User ID'].count()
print(user_growth_by_year)

Year
2021      14
2022    2448
2023      38
Name: User ID, dtype: int64
